In [1]:
import csv
import numpy as np
import os
import pandas as pd
import scipy.interpolate
import sklearn.metrics
import sys
sys.path.append("../src")
import localmodule


if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO


from matplotlib import pyplot as plt
%matplotlib inline

# Define constants.
dataset_name = localmodule.get_dataset_name()
models_dir = localmodule.get_models_dir()
units = localmodule.get_units()
n_units = len(units)
n_trials = 10


In [7]:
n_val_trials = 10

# Loop over augmentations.
for aug_kind_str in ["none", "all-but-noise", "all"]:
    
    # Loop over models.
    for model_name in ["icassp-ntt-convnet", "icassp-add-convnet", "pcen-convnet"]:


        if not aug_kind_str == "none":
            model_name = "_".join([model_name, "aug-" + aug_kind_str])
        model_dir = os.path.join(models_dir, model_name)

        for test_unit_id in range(6):

            test_unit_str = units[test_unit_id]
            test_unit_dir = os.path.join(model_dir, test_unit_str)

            val_accs = []
            for trial_id in range(n_trials):
                trial_str = "trial-" + str(trial_id)
                trial_dir = os.path.join(test_unit_dir, trial_str)
                history_name = "_".join([
                    dataset_name,
                    model_name,
                    test_unit_str,
                    trial_str,
                    "history.csv"
                ])
                history_path = os.path.join(
                    trial_dir, history_name)
                try:
                    history_df = pd.read_csv(history_path)
                    val_acc = max(history_df["Validation accuracy (%)"])
                except:
                    val_acc = 0.0
                val_accs.append(val_acc)

            val_accs = np.array(val_accs)
            best_trial_ids = np.argsort(val_accs)[::-1]
            best_trial_ids = best_trial_ids[:n_val_trials]
            
            try:

                for trial_id in best_trial_ids:

                    trial_str = "trial-" + str(trial_id)
                    fns, fps, tns, tps, accs = [], [], [], [], []
                    validation_units = localmodule.fold_units()[test_unit_id][2]

                    for val_unit_str in validation_units:

                        trial_dir = os.path.join(
                            test_unit_dir, trial_str)
                        predictions_name = "_".join([
                            dataset_name,
                            model_name,
                            "test-" + test_unit_str,
                            trial_str,
                            "predict-" + val_unit_str,
                            "clip-predictions.csv"
                        ])
                        prediction_path = os.path.join(
                            trial_dir, predictions_name)

                        # Load prediction.
                        with open(prediction_path, 'r') as f:
                            reader = csv.reader(f)
                            rows = list(reader)
                        rows = [",".join(row) for row in rows]
                        rows = rows[1:]
                        rows = "\n".join(rows)

                        # Parse rows with correct header.
                        df = pd.read_csv(StringIO(rows),
                            names=[
                                "Dataset",
                                "Test unit",
                                "Prediction unit",
                                "Timestamp",
                                "Center Freq (Hz)",
                                "Augmentation",
                                "Key",
                                "Ground truth",
                                "Predicted probability"])
                        y_pred = np.array(df["Predicted probability"])
                        y_pred = (y_pred > 0.5).astype('int')

                        # Load ground truth.
                        y_true = np.array(df["Ground truth"])

                        # Compute confusion matrix.
                        tn, fp, fn, tp = sklearn.metrics.confusion_matrix(
                            y_true, y_pred).ravel()

                        tns.append(tn)
                        fps.append(fp)
                        fns.append(fn)
                        tps.append(tp)

                    fn = sum(fns)
                    fp = sum(fps)
                    tn = sum(tns)
                    tp = sum(tps)

                    print(model_name, "{:5.2f}%".format(100 * (tn+tp) / (tn+tp+fn+fp)))
                    
            except:
                pass

            print("")
        print("")

icassp-ntt-convnet 95.99%
icassp-ntt-convnet 96.93%
icassp-ntt-convnet 97.06%
icassp-ntt-convnet 97.76%
icassp-ntt-convnet 95.61%
icassp-ntt-convnet 97.16%
icassp-ntt-convnet 91.81%
icassp-ntt-convnet 94.29%
icassp-ntt-convnet 83.06%
icassp-ntt-convnet 50.00%

icassp-ntt-convnet 86.95%
icassp-ntt-convnet 91.58%
icassp-ntt-convnet 84.87%
icassp-ntt-convnet 85.38%
icassp-ntt-convnet 80.61%
icassp-ntt-convnet 87.09%
icassp-ntt-convnet 85.06%
icassp-ntt-convnet 78.78%
icassp-ntt-convnet 87.47%
icassp-ntt-convnet 85.20%

icassp-ntt-convnet 80.49%
icassp-ntt-convnet 72.93%
icassp-ntt-convnet 80.74%
icassp-ntt-convnet 85.09%
icassp-ntt-convnet 83.76%
icassp-ntt-convnet 80.78%
icassp-ntt-convnet 78.66%
icassp-ntt-convnet 80.66%
icassp-ntt-convnet 74.41%
icassp-ntt-convnet 50.00%

icassp-ntt-convnet 97.71%
icassp-ntt-convnet 96.63%
icassp-ntt-convnet 96.19%
icassp-ntt-convnet 95.44%
icassp-ntt-convnet 95.28%
icassp-ntt-convnet 92.93%
icassp-ntt-convnet 50.00%
icassp-ntt-convnet 71.65%
icassp-nt

['unit07', 'unit10']

In [56]:
validation_units

['unit07', 'unit10']